In [1]:
import numpy as np
import pandas_datareader as pdr
import datetime
import pandas as pd
from datetime import datetime

In [2]:
from pandas_datareader.google.daily import GoogleDailyReader
@property
def url(self):
    return 'http://finance.google.com/finance/historical'
GoogleDailyReader.url = url

In [3]:
start = datetime(2007,1,1)
end = datetime(2014,1,1)
ret = pdr.get_data_google(['SPY'], start, end)
ret.to_frame().head()

,,Open,High,Low,Close,Volume
Date,minor,,,,,
2007-01-03,SPY,142.25,142.86,140.57,141.37,89183100.0
2007-01-04,SPY,141.23,142.05,140.61,141.67,66119200.0
2007-01-05,SPY,141.33,141.40,140.38,140.54,68523700.0
2007-01-08,SPY,140.82,141.41,140.25,141.19,66209700.0
2007-01-09,SPY,141.31,141.60,140.40,141.07,69505500.0


In [7]:
def get(tickers, startdate, enddate):
    def data(ticker):
        return pdr.get_data_google(ticker, start=startdate, end=enddate)

    datas = map(data, tickers)
    return pd.concat(datas, keys=tickers, names=['Ticker', 'Date'])

all_data = get(['AAPL', 'IBM', 'GOOG', 'SPY'], datetime(2007, 1, 1), datetime(2017, 1, 1))

In [8]:
all_data.drop('Volume', inplace=True, axis=1)

In [9]:
all_data.head(2)

Open   High    Low  Close
Ticker Date                                  
AAPL   2007-01-03  12.33  12.37  11.70  11.97
       2007-01-04  12.01  12.28  11.97  12.24

In [10]:
all_data.tail(5)

Open    High     Low   Close
Ticker Date                                      
SPY    2016-12-23  225.43  225.72  225.21  225.71
       2016-12-27  226.02  226.73  226.00  226.27
       2016-12-28  226.57  226.59  224.27  224.40
       2016-12-29  224.48  224.89  223.84  224.35
       2016-12-30  224.73  224.83  222.73  223.53

In [18]:
all_data.loc['AAPL'].iloc[20:30]

,Open,High,Low,Close
Date,,,,
2007-02-01,12.32,12.32,12.11,12.11
2007-02-02,12.02,12.18,11.96,12.11
2007-02-05,12.04,12.18,11.99,11.99
2007-02-06,12.06,12.07,11.84,12.02
2007-02-07,12.07,12.34,11.94,12.31
2007-02-08,12.20,12.36,12.20,12.31
2007-02-09,12.27,12.31,11.89,11.90
2007-02-12,12.06,12.17,11.95,12.13
2007-02-13,12.17,12.18,12.04,12.09


In [19]:
str(all_data.loc['AAPL'].iloc[20].name)

'2007-02-01 00:00:00'

In [20]:
all_data.loc['AAPL']['2007-01-03':'2007-01-11']['Close']

Date
2007-01-03    11.97
2007-01-04    12.24
2007-01-05    12.15
2007-01-08    12.21
2007-01-09    13.22
2007-01-10    13.86
2007-01-11    13.69
Name: Close, dtype: float64

In [21]:
all_data_flat = all_data.reset_index()

In [22]:
all_data_flat.head(2)

,Ticker,Date,Open,High,Low,Close
0,AAPL,2007-01-03,12.33,12.37,11.70,11.97
1,AAPL,2007-01-04,12.01,12.28,11.97,12.24


In [23]:
all_data_flat.pivot(index='Date', columns='Ticker', values='Close').head(2)

Ticker,AAPL,GOOG,IBM
Date,,,
2007-01-03,11.97,233.56,97.27
2007-01-04,12.24,241.39,98.31


In [24]:
all_data_unstacked = all_data.unstack(level=0)
all_data_unstacked.head(2)

Open                  High                   Low                 \
Ticker       AAPL    IBM    GOOG   AAPL    IBM    GOOG   AAPL    IBM    GOOG   
Date                                                                           
2007-01-03  12.33  97.18  232.77  12.37  98.40  238.09  11.70  96.26  230.32   
2007-01-04  12.01  97.25  234.27  12.28  98.79  241.73  11.97  96.88  233.94   

            Close                 
Ticker       AAPL    IBM    GOOG  
Date                              
2007-01-03  11.97  97.27  233.56  
2007-01-04  12.24  98.31  241.39

In [104]:
all_data_unstacked['Close', 'AAPL'].head(2)

Date
2007-01-03    11.97
2007-01-04    12.24
Name: (Close, AAPL), dtype: float64

In [94]:
all_data_unstacked_pct = all_data_unstacked.pct_change().fillna(0)
all_data_unstacked_pct.head(2)

Open                         High                      \
Ticker          AAPL      IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                    
2007-01-03  0.000000  0.00000  0.000000  0.000000  0.000000  0.000000   
2007-01-04 -0.025953  0.00072  0.006444 -0.007276  0.003963  0.015288   

                 Low                         Close                      \
Ticker          AAPL       IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                     
2007-01-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2007-01-04  0.023077  0.006441  0.015717  0.022556  0.010692  0.033525   

              Volume                 
Ticker          AAPL       IBM GOOG  
Date                                 
2007-01-03  0.000000  0.000000  0.0  
2007-01-04 -0.312753  0.147584  0.0

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
rows = all_data_unstacked_pct.shape[0]
train_size = int(rows*0.75)
test_size = rows-train_size
print('Train: %d, test: %d' % (train_size, test_size))

Train: 1888, test: 630


In [12]:
train_X = all_data_unstacked_pct.iloc[0:train_size]
test_X = all_data_unstacked_pct.iloc[train_size:]
assert train_X.shape[0] + test_X.shape[0] == rows

In [13]:
train_X.head()

Open                          High                      \
Ticker          AAPL       IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                     
2007-01-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2007-01-04 -0.025953  0.000720  0.006444 -0.007276  0.003963  0.015288   
2007-01-05  0.019983  0.003599  0.028770  0.002443 -0.008503  0.007364   
2007-01-08  0.002449  0.009221  0.010746  0.004062  0.015824  0.004846   
2007-01-09  0.005700  0.005888 -0.004598  0.074434  0.008342 -0.003310   

                 Low                         Close                      \
Ticker          AAPL       IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                     
2007-01-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2007-01-04  0.023077  0.006441  0.015717  0.022556  0.010692  0.033525   
2007-01-05  0.007519  0.000310  0.020860 -0.007353 -0.009053  0.008120   
2007-01-08  0.009950  0.014859  0.008542  0.004938  0.015192 -0.007397   
2007-01-09 -0.001642  0.007321 -0.002076  0.082719  0.011830  0.003974   

              Volume                 
Ticker          AAPL       IBM GOOG  
Date                                 
2007-01-03  0.000000  0.000000  0.0  
2007-01-04 -0.312753  0.147584  0.0  
2007-01-05 -0.024363 -0.315832  0.0  
2007-01-08 -0.044947  0.431572  0.0  
2007-01-09  3.202131  0.074351  0.0

In [14]:
from sklearn.preprocessing import StandardScaler

In [63]:
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

In [78]:
train_X_scaled = pd.DataFrame(data=train_X_scaled, columns=train_X.columns, index=train_X.index)
test_X_scaled = pd.DataFrame(data=test_X_scaled, columns=test_X.columns, index=test_X.index)
train_X_scaled.head()

Open                          High                      \
Ticker          AAPL       IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                     
2007-01-03 -0.056854 -0.031304 -0.028985 -0.067191 -0.034376 -0.036396   
2007-01-04 -1.141226  0.019096  0.169111 -0.459282  0.289591  0.860756   
2007-01-05  0.778095  0.220515  0.855429  0.064464 -0.729396  0.395715   
2007-01-08  0.045470  0.613910  0.301367  0.151699  1.259100  0.247966   
2007-01-09  0.181319  0.380701 -0.170321  3.944098  0.647470 -0.230652   

                 Low                         Close                      \
Ticker          AAPL       IBM      GOOG      AAPL       IBM      GOOG   
Date                                                                     
2007-01-03 -0.059790 -0.032351 -0.035378 -0.060116 -0.031200 -0.034167   
2007-01-04  0.955350  0.436986  0.800880  0.953554  0.697625  1.648268   
2007-01-05  0.270957 -0.009787  1.074509 -0.390553 -0.648307  0.373318   
2007-01-08  0.377915  1.050411  0.419110  0.161807  1.004377 -0.405374   
2007-01-09 -0.132023  0.501103 -0.145829  3.657230  0.775215  0.165285   

              Volume                      
Ticker          AAPL       IBM      GOOG  
Date                                      
2007-01-03 -0.136343 -0.146259 -0.021249  
2007-01-04 -0.985940  0.238283 -0.021249  
2007-01-05 -0.202526 -0.969182 -0.021249  
2007-01-08 -0.258443  0.978234 -0.021249  
2007-01-09  8.562293  0.047469 -0.021249

In [17]:
train_X_scaled.describe()

Open                                      High                \
Ticker          AAPL           IBM          GOOG          AAPL           IBM   
count   1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03   
mean    6.615471e-18  4.586726e-18  1.709732e-17  1.308393e-18  1.511268e-17   
std     1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00   
min    -6.113093e+00 -6.917997e+00 -1.525398e+01 -6.788799e+00 -5.891316e+00   
25%    -4.933883e-01 -4.569576e-01 -2.836463e-01 -4.923048e-01 -4.557095e-01   
50%     1.381718e-02  1.822354e-02 -8.202510e-03 -6.229087e-03 -3.007348e-05   
75%     5.054681e-01  5.256086e-01  2.860741e-01  4.807749e-01  4.355084e-01   
max     9.327024e+00  5.418440e+00  3.125474e+01  6.747618e+00  6.395425e+00   

                               Low                                     Close  \
Ticker          GOOG          AAPL           IBM          GOOG          AAPL   
count   1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03   
mean   -4.777840e-18 -3.733331e-17 -4.116293e-17  1.017312e-17  3.969283e-19   
std     1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00   
min    -5.316449e+00 -8.813099e+00 -6.424892e+00 -5.130838e+00 -8.106035e+00   
25%    -4.393732e-01 -4.507512e-01 -4.166981e-01 -4.511574e-01 -4.878459e-01   
50%    -2.203250e-02  3.089916e-02  1.859382e-02  2.704752e-02  4.863213e-04   
75%     4.511142e-01  4.517891e-01  4.595603e-01  4.667880e-01  5.236372e-01   
max     1.124686e+01  8.410895e+00  6.179507e+00  9.287179e+00  6.290016e+00   

                                          Volume                              
Ticker           IBM          GOOG          AAPL           IBM          GOOG  
count   1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03  1.888000e+03  
mean    2.528580e-18 -1.411300e-18 -2.469776e-18 -2.128712e-17  6.381430e-16  
std     1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00  1.000265e+00  
min    -5.674692e+00 -5.859183e+00 -2.052510e+00 -2.291763e+00 -9.532627e+00  
25%    -4.717483e-01 -4.416886e-01 -6.343269e-01 -6.055594e-01 -2.124889e-02  
50%    -8.295980e-03 -2.090848e-02 -1.772898e-01 -1.761633e-01 -2.124889e-02  
75%     5.094054e-01  4.582160e-01  4.148680e-01  3.724554e-01 -2.124889e-02  
max     7.818135e+00  9.998386e+00  1.369210e+01  1.253017e+01  2.070746e+01

In [18]:
train_X_scaled.max()

        Ticker
Open    AAPL       9.327024
        IBM        5.418440
        GOOG      31.254744
High    AAPL       6.747618
        IBM        6.395425
        GOOG      11.246862
Low     AAPL       8.410895
        IBM        6.179507
        GOOG       9.287179
Close   AAPL       6.290016
        IBM        7.818135
        GOOG       9.998386
Volume  AAPL      13.692099
        IBM       12.530170
        GOOG      20.707462
dtype: float64

In [85]:
idx = pd.IndexSlice
aapl = train_X_scaled.loc[:, idx[:, 'AAPL']]
aapl.head()

,Open,High,Low,Close,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2007-01-03,-0.056854,-0.067191,-0.059790,-0.060116,-0.136343
2007-01-04,-1.141226,-0.459282,0.955350,0.953554,-0.985940
2007-01-05,0.778095,0.064464,0.270957,-0.390553,-0.202526
2007-01-08,0.045470,0.151699,0.377915,0.161807,-0.258443
2007-01-09,0.181319,3.944098,-0.132023,3.657230,8.562293


In [90]:
aapl.columns = aapl.columns.droplevel(1)

In [92]:
aapl.head()

,Open,High,Low,Close,Volume
Date,,,,,
2007-01-03,-0.056854,-0.067191,-0.059790,-0.060116,-0.136343
2007-01-04,-1.141226,-0.459282,0.955350,0.953554,-0.985940
2007-01-05,0.778095,0.064464,0.270957,-0.390553,-0.202526
2007-01-08,0.045470,0.151699,0.377915,0.161807,-0.258443
2007-01-09,0.181319,3.944098,-0.132023,3.657230,8.562293


In [77]:
aapl['Close'].shape

(1888, 1)